In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import itertools
import random
from statannot import add_stat_annotation

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import itertools
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler

In [ ]:
import lifelines
import seaborn as sns
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
from lifelines.plotting import add_at_risk_counts

In [ ]:
h_cluster=pd.read_csv("/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/6/H/6/cluster_x_column.tsv", sep='\t', index_col=0)
basal_information=pd.read_csv('/Volumes/SSD_Yue/TCGA_Data/PAM50CLOW.csv')
list_basal=basal_information[basal_information.newSUBTYPE=='Basal'].iloc[:,0].tolist()

In [ ]:
#We choose to use this way to assign samples to the states so that it is consistent with the state map (onco-gps)
list_F0 = h_cluster.T[h_cluster.T.C0==1].index.tolist() #F0 n=53
list_F1 = h_cluster.T[h_cluster.T.C4==1].index.tolist() #F1 n=18
list_F2 = h_cluster.T[h_cluster.T.C3==1].index.tolist() #F2 n=21
list_F3 = h_cluster.T[h_cluster.T.C1==1].index.tolist() #F3 n=36
list_F5 = h_cluster.T[h_cluster.T.C2==1].index.tolist() #F5 n=22
list_F1_and_F2_mixture = h_cluster.T[h_cluster.T.C5==1].index.tolist() #F1_and_F2_mixture n=35
print("F0",len(list_F0))
print("F1",len(list_F1))
print("F2",len(list_F2))
print("F3",len(list_F3))
print("F5",len(list_F5))
print("F1_and_F2_mixture",len(list_F1_and_F2_mixture))

In [ ]:
dict_cell_lines={}
dict_cell_lines["F0"]=list_F0
dict_cell_lines["F1"]=list_F1
dict_cell_lines["F2"]=list_F2
dict_cell_lines["F3"]=list_F3
dict_cell_lines["F5"]=list_F5
dict_cell_lines["F1_and_F2_mixture"]=list_F1_and_F2_mixture

In [ ]:
patient_to_state = {patient: state for state, patients in dict_cell_lines.items() for patient in patients}

In [ ]:
#patients from different states and truncating the OS_months to 120 months
df_os=pd.read_csv('../../../Downloads/KM_Plot__Overall_Survival__(months).txt', sep='\t')
df_os['Event']=[i.split(':')[0] for i in df_os['OS_STATUS']]
df_os_tnbc=df_os[df_os['Patient ID'].isin(list_basal)]
truncated_os_months = df_os_tnbc['OS_MONTHS'].clip(upper=120)
df_os_tnbc['OS_months_truncated'] = truncated_os_months
df_os_tnbc['event_truncated'] = df_os_tnbc['Event'].where(df_os_tnbc['OS_MONTHS'] <= 120, 0)
df_os_tnbc['group'] = df_os_tnbc['Patient ID'].map(patient_to_state)

In [ ]:
# Initialize the Kaplan-Meier fitter
kmf = KaplanMeierFitter()

# Plot the KM curve for each group
plt.figure(figsize=(10, 6))
for label in df_os_tnbc['group'].unique():
    mask = df_os_tnbc['group'] == label
    kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
    median_survival = kmf.median_survival_time_
    kmf.plot(ci_show=False)
    #print(label, median_survival)
    #survival_table = kmf.event_table
    #print(survival_table)
    #add_at_risk_counts(kmf)


# Customize plot
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")
#plt.savefig('/Volumes/SSD_Yue/TNBC_paper_Fig1_11252024/KM_plot_TCGA_different_states_OS_truncated_to_120_months_11262024.svg')


In [ ]:
#To print survival table for each identity
label="F0"
mask = df_os_tnbc['group'] == label
kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
median_survival = kmf.median_survival_time_
kmf.plot(ci_show=False)
print(label, median_survival)
survival_table = kmf.event_table
print(survival_table)
add_at_risk_counts(kmf)
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")

In [ ]:
#To print survival table for each identity
label="F1"
mask = df_os_tnbc['group'] == label
kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
median_survival = kmf.median_survival_time_
kmf.plot(ci_show=False)
print(label, median_survival)
survival_table = kmf.event_table
print(survival_table)
add_at_risk_counts(kmf)
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")

In [ ]:
#To print survival table for each identity
label="F2"
mask = df_os_tnbc['group'] == label
kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
median_survival = kmf.median_survival_time_
kmf.plot(ci_show=False)
print(label, median_survival)
survival_table = kmf.event_table
print(survival_table)
add_at_risk_counts(kmf)
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")

In [ ]:
#To print survival table for each identity
label="F3"
mask = df_os_tnbc['group'] == label
kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
median_survival = kmf.median_survival_time_
kmf.plot(ci_show=False)
print(label, median_survival)
survival_table = kmf.event_table
print(survival_table)
add_at_risk_counts(kmf)
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")

In [ ]:
#To print survival table for each identity
label="F5"
mask = df_os_tnbc['group'] == label
kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
median_survival = kmf.median_survival_time_
kmf.plot(ci_show=False)
print(label, median_survival)
survival_table = kmf.event_table
print(survival_table)
add_at_risk_counts(kmf)
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")

In [ ]:
#To print survival table for each identity
label="F1_and_F2_mixture"
mask = df_os_tnbc['group'] == label
kmf.fit(df_os_tnbc.loc[mask, 'OS_months_truncated'], event_observed=df_os_tnbc.loc[mask, 'event_truncated'], label=label)
median_survival = kmf.median_survival_time_
kmf.plot(ci_show=False)
print(label, median_survival)
survival_table = kmf.event_table
print(survival_table)
add_at_risk_counts(kmf)
plt.title("Kaplan-Meier Plot Based on Gene Expression")
plt.xlabel("Time")
plt.ylabel("Survival Probability")
plt.legend(title="Expression Group")

In [ ]:
high_group = df_os_tnbc[df_os_tnbc['group'] == 'F5']
low_group = df_os_tnbc[df_os_tnbc['group'] == 'F1_and_F2_mixture']
# Perform the log-rank test
results = logrank_test(
    high_group['OS_months_truncated'], low_group['OS_months_truncated'],
    event_observed_A=high_group['event_truncated'], event_observed_B=low_group['event_truncated']
)

# Print the p-value
print("Log-rank test p-value:", results.p_value)